In [1]:
import os
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv

In [2]:
load_dotenv()

# config and credentials
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('username')
password = os.getenv('password')

# connection
engine = db.create_engine(
    'mssql://{}:{}@{}/{}?driver=ODBC+Driver+18+for+SQL+Server'.format(
        username, password, server, database
    )
)

# establish connection
connection = engine.connect()

### CSAD Bonds

In [3]:
query = '''
    SELECT
        Date,
        Rm,
        ABS(Rm) AS AbsoluteRm,
        POWER(Rm, 2) AS SquaredRm,
        Sum / Count AS Csad
    FROM (
        SELECT
            Date,
            Rm,
            ABS(SUM(RetEom) - Rm) AS Sum,
            COUNT(DISTINCT Cusip) AS Count
        FROM (
            SELECT
                A.TrdExctnDtEOM AS Date,
                A.CusipId AS Cusip,
                A.R AS RetEom,
                B.Rm
            FROM
                BondReturns_customerOnly A
            INNER JOIN (
                SELECT
                    TrdExctnDtEOM AS Date,
                    SUM(R * TD_volume) / SUM(TD_volume) AS Rm
                FROM
                    BondReturns_customerOnly
                GROUP BY
                    TrdExctnDtEOM
            ) B ON A.TrdExctnDtEOM = B.Date
        ) A
        GROUP BY
            Date,
            Rm
    ) B
    WHERE
	    Rm IS NOT NULL
    ORDER BY
        Date
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/csad-bonds-monthly.csv', index=False)

In [4]:
query = '''
    SELECT
        Date,
        Rm,
        ABS(Rm) AS AbsoluteRm,
        POWER(Rm, 2) AS SquaredRm,
        Sum / Count AS Csad
    FROM (	
        SELECT
            Date,
            Rm,
            ABS(SUM(RetEom) - Rm) AS Sum,
            COUNT(DISTINCT Cusip) AS Count
        FROM (	
            SELECT
                A.*,
                B.Rm
            FROM (
                SELECT
                    Date,
                    Cusip,
                    EXP(SUM(LOG(RetEom))) - 1 AS RetEom
                FROM (
                    SELECT
                        A.TrdExctnDtEOM,
                        CASE 
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 1 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 01, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 2 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 04, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 3 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 07, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 4 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 10, 01)
                        END AS Date,
                        A.CusipId AS Cusip,
                        A.R + 1 AS RetEom
                    FROM
                        BondReturns_customerOnly A
                ) A
                GROUP BY
                    A.Date,
                    A.Cusip
            ) A
            INNER JOIN (
                SELECT
                    Date,
                    EXP(SUM(LOG(Rm))) - 1 AS Rm
                FROM (
                    SELECT
                        TrdExctnDtEOM,
                        CASE 
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 1 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 01, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 2 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 04, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 3 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 07, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 4 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 10, 01)
                        END AS Date,
                        SUM(R * TD_volume) / SUM(TD_volume) + 1 AS Rm
                    FROM
                        BondReturns_customerOnly
                    GROUP BY
                        TrdExctnDtEOM
                ) A
                GROUP BY
                    Date
            ) B ON A.Date = B.Date
        ) A
        GROUP BY
            Date,
            Rm
    ) B
    WHERE
        Rm IS NOT NULL
    ORDER BY
        Date
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/csad-bonds-quarterly.csv', index=False)

### Institutional

In [5]:
query = '''
    SELECT
        Date,
        Rm,
        ABS(Rm) AS AbsoluteRm,
        POWER(Rm, 2) AS SquaredRm,
        Sum / Count AS Csad
    FROM (
        SELECT
            Date,
            Rm,
            ABS(SUM(RetEom) - Rm) AS Sum,
            COUNT(DISTINCT Cusip) AS Count
        FROM (
            SELECT
                A.TrdExctnDtEOM AS Date,
                A.CusipId AS Cusip,
                A.R AS RetEom,
                B.Rm
            FROM
                BondReturns_customerOnly_institutional A
            INNER JOIN (
                SELECT
                    TrdExctnDtEOM AS Date,
                    SUM(R * TD_volume) / SUM(TD_volume) AS Rm
                FROM
                    BondReturns_customerOnly_institutional
                GROUP BY
                    TrdExctnDtEOM
            ) B ON A.TrdExctnDtEOM = B.Date
        ) A
        GROUP BY
            Date,
            Rm
    ) B
    WHERE
	    Rm IS NOT NULL
    ORDER BY
        Date
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/csad-bonds-institutional-monthly.csv', index=False)

In [6]:
query = '''
    SELECT
        Date,
        Rm,
        ABS(Rm) AS AbsoluteRm,
        POWER(Rm, 2) AS SquaredRm,
        Sum / Count AS Csad
    FROM (	
        SELECT
            Date,
            Rm,
            ABS(SUM(RetEom) - Rm) AS Sum,
            COUNT(DISTINCT Cusip) AS Count
        FROM (	
            SELECT
                A.*,
                B.Rm
            FROM (
                SELECT
                    Date,
                    Cusip,
                    EXP(SUM(LOG(RetEom))) - 1 AS RetEom
                FROM (
                    SELECT
                        A.TrdExctnDtEOM,
                        CASE 
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 1 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 01, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 2 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 04, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 3 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 07, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 4 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 10, 01)
                        END AS Date,
                        A.CusipId AS Cusip,
                        A.R + 1 AS RetEom
                    FROM
                        BondReturns_customerOnly_institutional A
                ) A
                GROUP BY
                    A.Date,
                    A.Cusip
            ) A
            INNER JOIN (
                SELECT
                    Date,
                    EXP(SUM(LOG(Rm))) - 1 AS Rm
                FROM (
                    SELECT
                        TrdExctnDtEOM,
                        CASE 
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 1 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 01, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 2 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 04, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 3 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 07, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 4 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 10, 01)
                        END AS Date,
                        SUM(R * TD_volume) / SUM(TD_volume) + 1 AS Rm
                    FROM
                        BondReturns_customerOnly_institutional
                    GROUP BY
                        TrdExctnDtEOM
                ) A
                GROUP BY
                    Date
            ) B ON A.Date = B.Date
        ) A
        GROUP BY
            Date,
            Rm
    ) B
    WHERE
        Rm IS NOT NULL
    ORDER BY
        Date
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/csad-bonds-institutional-quarterly.csv', index=False)

### Retail

In [7]:
query = '''
    SELECT
        Date,
        Rm,
        ABS(Rm) AS AbsoluteRm,
        POWER(Rm, 2) AS SquaredRm,
        Sum / Count AS Csad
    FROM (
        SELECT
            Date,
            Rm,
            ABS(SUM(RetEom) - Rm) AS Sum,
            COUNT(DISTINCT Cusip) AS Count
        FROM (
            SELECT
                A.TrdExctnDtEOM AS Date,
                A.CusipId AS Cusip,
                A.R AS RetEom,
                B.Rm
            FROM
                BondReturns_customerOnly_retail A
            INNER JOIN (
                SELECT
                    TrdExctnDtEOM AS Date,
                    SUM(R * TD_volume) / SUM(TD_volume) AS Rm
                FROM
                    BondReturns_customerOnly_retail
                GROUP BY
                    TrdExctnDtEOM
            ) B ON A.TrdExctnDtEOM = B.Date
        ) A
        GROUP BY
            Date,
            Rm
    ) B
    WHERE
	    Rm IS NOT NULL
    ORDER BY
        Date
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/csad-bonds-retail-monthly.csv', index=False)

In [8]:
query = '''
    SELECT
        Date,
        Rm,
        ABS(Rm) AS AbsoluteRm,
        POWER(Rm, 2) AS SquaredRm,
        Sum / Count AS Csad
    FROM (	
        SELECT
            Date,
            Rm,
            ABS(SUM(RetEom) - Rm) AS Sum,
            COUNT(DISTINCT Cusip) AS Count
        FROM (	
            SELECT
                A.*,
                B.Rm
            FROM (
                SELECT
                    Date,
                    Cusip,
                    EXP(SUM(LOG(RetEom))) - 1 AS RetEom
                FROM (
                    SELECT
                        A.TrdExctnDtEOM,
                        CASE 
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 1 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 01, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 2 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 04, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 3 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 07, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 4 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 10, 01)
                        END AS Date,
                        A.CusipId AS Cusip,
                        A.R + 1 AS RetEom
                    FROM
                        BondReturns_customerOnly_retail A
                ) A
                GROUP BY
                    A.Date,
                    A.Cusip
            ) A
            INNER JOIN (
                SELECT
                    Date,
                    EXP(SUM(LOG(Rm))) - 1 AS Rm
                FROM (
                    SELECT
                        TrdExctnDtEOM,
                        CASE 
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 1 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 01, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 2 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 04, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 3 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 07, 01)
                            WHEN DATEPART(QUARTER, TrdExctnDtEOM) = 4 THEN DATEFROMPARTS(YEAR(TrdExctnDtEOM), 10, 01)
                        END AS Date,
                        SUM(R * TD_volume) / SUM(TD_volume) + 1 AS Rm
                    FROM
                        BondReturns_customerOnly_retail
                    GROUP BY
                        TrdExctnDtEOM
                ) A
                GROUP BY
                    Date
            ) B ON A.Date = B.Date
        ) A
        GROUP BY
            Date,
            Rm
    ) B
    WHERE
        Rm IS NOT NULL
    ORDER BY
        Date
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/csad-bonds-retail-quarterly.csv', index=False)